<a href="https://colab.research.google.com/github/akkipant/CS5542-Big-Data-Analytics-and-Applications/blob/master/ICP-6/Source/BDA_ICP_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

Read the data

First, look in the text:

In [3]:
# Read, then decode for py2 compat.
text = open('story.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 6053792 characters


In [4]:
text = text.lower() #Lowered the text otherwise model will treat lowercase and uppercase as different

In [6]:
# Take a look at the first 250 characters in text
print(text[:250])

thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country . families of soldiers killed in the conflict joined the protesters who carried banners with such slogans 


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

61 unique characters


Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [10]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [63]:
char2idx

{' ': 0,
 '!': 1,
 '"': 2,
 '#': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 '`': 32,
 'a': 33,
 'b': 34,
 'c': 35,
 'd': 36,
 'e': 37,
 'f': 38,
 'g': 39,
 'h': 40,
 'i': 41,
 'j': 42,
 'k': 43,
 'l': 44,
 'm': 45,
 'n': 46,
 'o': 47,
 'p': 48,
 'q': 49,
 'r': 50,
 's': 51,
 't': 52,
 'u': 53,
 'v': 54,
 'w': 55,
 'x': 56,
 'y': 57,
 'z': 58,
 '~': 59,
 '�': 60}

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [46]:
# The maximum length sentence we want for a single input in characters
seq_length = 30
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

t
h
o
u
s


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [47]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'thousands of demonstrators have'
' marched through london to prot'
'est the war in iraq and demand '
'the withdrawal of british troop'
's from that country . families '


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [48]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [49]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'thousands of demonstrators hav'
Target data: 'housands of demonstrators have'


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [50]:
# Batch size
BATCH_SIZE = 128

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((128, 30), (128, 30)), types: (tf.int64, tf.int64)>

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.GRU: A type of RNN with size units=rnn_units (You can also use a LSTM layer here.)

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [52]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Using an LSTM instead of a GRU

In [53]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         activation='tanh', recurrent_activation='sigmoid',
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(rnn_units,
                         activation='tanh', recurrent_activation='sigmoid',
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [54]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [55]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(128, 30, 61) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 30 but the model can be run on inputs of any length:

In [57]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (128, None, 256)          15616     
_________________________________________________________________
lstm_7 (LSTM)                (128, None, 1024)         5246976   
_________________________________________________________________
dropout_7 (Dropout)          (128, None, 1024)         0         
_________________________________________________________________
lstm_8 (LSTM)                (128, None, 1024)         8392704   
_________________________________________________________________
dropout_8 (Dropout)          (128, None, 1024)         0         
_________________________________________________________________
dense_4 (Dense)              (128, None, 61)           62525     
Total params: 13,717,821
Trainable params: 13,717,821
Non-trainable params: 0
__________________________________________

To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [58]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([59,  8, 48, 31, 18, 53, 20, 14, 22,  8, 33, 23, 25, 23, 25,  2, 50,
       36, 26, 54, 54,  9, 56, 12, 32, 40, 32,  1,  0,  4])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [59]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (128, 30, 61)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1106563


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [60]:
model.compile(optimizer='RMSprop', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [61]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 10 epochs to train the model. 

In [62]:
EPOCHS=5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
1525/1525 [==============================] - 174s 114ms/step - loss: 1.6912
Epoch 2/5
1525/1525 [==============================] - 173s 114ms/step - loss: 1.3130
Epoch 3/5
1525/1525 [==============================] - 173s 113ms/step - loss: 1.2555
Epoch 4/5
1525/1525 [==============================] - 172s 113ms/step - loss: 1.2244
Epoch 5/5
1525/1525 [==============================] - 172s 113ms/step - loss: 1.2029


Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [66]:
tf.train.latest_checkpoint(checkpoint_dir)
model_pred = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model_pred.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model_pred.build(tf.TensorShape([1, None]))
model_pred.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 256)            15616     
_________________________________________________________________
lstm_9 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dropout_9 (Dropout)          (1, None, 1024)           0         
_________________________________________________________________
lstm_10 (LSTM)               (1, None, 1024)           8392704   
_________________________________________________________________
dropout_10 (Dropout)         (1, None, 1024)           0         
_________________________________________________________________
dense_5 (Dense)              (1, None, 61)             62525     
Total params: 13,717,821
Trainable params: 13,717,821
Non-trainable params: 0
__________________________________________

The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [67]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 3000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [75]:
print(generate_text(model_pred, start_string=u"today"))

today 's hanging , from pro-hease has covered a new explosives described the condition taleban militants intaining emergency and kaso in june and the rebel police attracted more than 70 others . at least 250 people dead . a sri lanka 's cahonday , she believe the government f the six islamic militants died from faman says emergency and provincial rebel groups , peter 's immediate , mr. zarimi repamengin of britain . a jihad come up attending conment on the memorial population of the facilities of congolese fled under free warranted bd a barrel for the ex-based militant . once egypt to provide return to eight indian region and power tuesday , mr. khakai is a ponto said clashes in the gaza strip . as president vladimir putin said about 500 kilometers early into the first week reporters that has been destroyed last week , a burma an " return to a common hit " says any dideat protesters are opened fire on the world 's lodies of u.s. investigation as koir better that would 516 in afghanon ,